In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
model_original=models.resnet50(pretrained=False)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
from torchsummary import summary
summary(model_original, (3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]           4,096
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11          [-1, 256, 16, 16]          16,384
      BatchNorm2d-12          [-1, 256, 16, 16]             512
           Conv2d-13          [-1, 256, 16, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [74]:
class BottleNeck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        expansion = 4
        
        super(BottleNeck, self).__init__()
        self.conv1=nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.b1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.b2=nn.BatchNorm2d(out_channels)
        self.conv3=nn.Conv2d(out_channels, out_channels*expansion, kernel_size=1, bias=False)
        self.b3=nn.BatchNorm2d(out_channels*expansion)

        self.relu=nn.ReLU()
        self.downsample=downsample
        self.stride=stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity=x
        out=self.conv1(x)
        out=self.b1(out)
        out=self.relu(out)
        out=self.conv2(out)
        out=self.b2(out)
        out=self.relu(out)
        out=self.conv3(out)
        out=self.b3(out)
        if self.downsample is not None:
            identity=self.downsample(x)
        out+=identity
        out=self.relu(out)
        return out
        
        

In [77]:
class TinyResNet(nn.Module):
    def __init__(self):
        super(TinyResNet, self).__init__()
        self.in_channels=64
        self.conv1=nn.Conv2d(3,64,kernel_size=7, stride=2, padding=3, bias=False)
        self.bn=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        downsample= nn.Sequential(nn.Conv2d(64,256,kernel_size=2, stride=2, bias=False),
                                 nn.BatchNorm2d(256))
        self.layer1=BottleNeck(64,64, stride=2, downsample=downsample)

    def forward(self, x):
        x=self.conv1(x)
        x=self.bn(x)
        x=self.relu(x)
        x=self.layer1(x)
        return x

In [78]:
from torchsummary import summary
import torch
import torch.nn as nn

# Assuming your model class TinyResNet is defined
model = TinyResNet()

# Move model to the device (CPU or CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print model summary for input size (e.g., RGB images 3x224x224)
summary(model, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
            Conv2d-4         [-1, 64, 112, 112]           4,096
       BatchNorm2d-5         [-1, 64, 112, 112]             128
              ReLU-6         [-1, 64, 112, 112]               0
            Conv2d-7           [-1, 64, 56, 56]          36,864
       BatchNorm2d-8           [-1, 64, 56, 56]             128
              ReLU-9           [-1, 64, 56, 56]               0
           Conv2d-10          [-1, 256, 56, 56]          16,384
      BatchNorm2d-11          [-1, 256, 56, 56]             512
           Conv2d-12          [-1, 256, 56, 56]          65,536
      BatchNorm2d-13          [-1, 256, 56, 56]             512
             ReLU-14          [-1, 256,